In [15]:
pip install --user lda

Note: you may need to restart the kernel to use updated packages.


In [47]:
#!/usr/bin/python3.6
#Install LDA library if not already installed
# pip3.6 install --user lda
# the input file is a two-column file yelp_reviews_short.xslx
# there are two output files: topic_word_dist.xlsx and document_topic_dist.xlsx
# the script prompts for the name of the columns -- Restaurant_name and Restaurant_review in the yelp_reviews_short.xslx file
#Revised by Aadit Barua
import numpy as np
import os, csv, nltk, lda
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [48]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
review_df = pd.read_pickle('president_df')
# reviews_df=pd.read_excel("yelp_reviews_short.xlsx",encoding='utf8', errors='ignore')
#reviews_list = reviews_df["review"].tolist()
#for i in range(0,len(reviews_list)):
#    reviews_list[i] = str(reviews_list[i])
#reviews_df['review'] = reviews_list
review_df['Speech'].iloc[0]
review_df.reset_index(inplace = True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
modi = pd.read_excel('modi.xlsx')
modi = modi.iloc[0:4,:]
modi.head()

,index,President,Date,Speech
0,N,Narendra Modi,2014-01-05,"After Independence, this is the first election..."
1,N,Narendra Modi,2013-12-22,Present on the stage National President of Bha...
2,N,Narendra Modi,2014-02-05,National president of BJP Shri Rajnath Singh J...
3,N,Narendra Modi,2014-02-02,"""When I was coming on the helicopter here, it ..."


In [50]:
review_df.head()
latest_presidents = ['Joe Biden', 'Donald Trump', 'Barack Obama', 'George W. Bush',
'Bill Clinton', 'George H. W. Bush']
review_df = review_df.loc[review_df['President'].isin(latest_presidents),:].copy()
review_df['President'].value_counts()

Barack Obama         50
Donald Trump         43
George W. Bush       39
Bill Clinton         39
George H. W. Bush    23
Joe Biden            10
Name: President, dtype: int64

In [51]:
#the above doesn't work as 'nan' is a string
review_df = review_df[review_df['Speech'] != 'nan']
review_df = pd.concat([modi,review_df], ignore_index=True)
review_df.head()

,index,President,Date,Speech
0,N,Narendra Modi,2014-01-05 00:00:00,"After Independence, this is the first election..."
1,N,Narendra Modi,2013-12-22 00:00:00,Present on the stage National President of Bha...
2,N,Narendra Modi,2014-02-05 00:00:00,National president of BJP Shri Rajnath Singh J...
3,N,Narendra Modi,2014-02-02 00:00:00,"""When I was coming on the helicopter here, it ..."
4,september-9-2021-remarks-fighting-covid-19-pan...,Joe Biden,"September 09, 2021",anscript\n \n\n\n\n\nTranscript\nTHE PRES...


In [52]:
import nltk

def noun(x):
    text = nltk.word_tokenize(x)
    pos_tagged = nltk.pos_tag(text)
    nouns = []
    for (i,j) in pos_tagged:
        if j == 'NN':
            nouns.append(i)
    return " ".join(nouns)

review_df['Speech'] = review_df['Speech'].map(noun)
review_df.head()

,index,President,Date,Speech
0,N,Narendra Modi,2014-01-05 00:00:00,election election movement election movement e...
1,N,Narendra Modi,2013-12-22 00:00:00,Present stage party dais ji belief role none m...
2,N,Narendra Modi,2014-02-05 00:00:00,president secretary stage .. gratitude gatheri...
3,N,Narendra Modi,2014-02-02 00:00:00,helicopter saffron sea mela sabha lot time tod...
4,september-9-2021-remarks-fighting-covid-19-pan...,Joe Biden,"September 09, 2021",anscript evening battle progress work variant ...


In [53]:
import re
review_df['Speech'] = review_df['Speech'].apply(lambda x : re.findall('[a-z]+',str(x)))

review_df['Speech'] = review_df['Speech'].apply(lambda x: ' '.join(x))
review_df['Speech'].iloc[0]

'election election movement election movement everything mission honour pain independence honour honour oday country honour country pessimist word pessimism dictionary dictionary life life country child tea train pessimist mother cleaning water hope cotton cotton home cotton cover cotton wage omeone life pessimism life omeone pain pain pain sensitivity strength feeling strength welfare country inspiration god way family shadow votebank plank development effort pressure advertisement airport night track record cent population nation world youth intelligence skill dividend asset disaster sin emphasis skill development outh way livelihood government skill development body vision track record area country productivity country country livestock rearing agriculture agriculture third third livestock rearing land farmer right area country timber government gold import gold smuggling business network period government gold smuggling kg gold operation treatment treatment animal bear pain cent ju

In [54]:
#restaurant_name = input('provide the column name for id: ')
#restaurant_review = input('provide the column name for text: ')
#ntopics= input('Provide the number of latent topics: ');

restaurant_name = 'President'
restaurant_review = 'Speech'
ntopics= '4'

In [55]:
word_tokenizer=RegexpTokenizer(r'[a-z]+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=list(set(stopwords.words('english'))) + ['austin', 'restaurant']


def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(review_df[restaurant_review])

print(total_features_words.shape)


(208, 6499)


C:\Users\chara\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [56]:
total_features_words

<208x6499 sparse matrix of type '<class 'numpy.int64'>'
	with 53930 stored elements in Compressed Sparse Row format>

In [57]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)


INFO:lda:n_documents: 208
INFO:lda:vocab_size: 6499
INFO:lda:n_words: 110343
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -976628
INFO:lda:<10> log likelihood: -879028
INFO:lda:<20> log likelihood: -841348
INFO:lda:<30> log likelihood: -830797
INFO:lda:<40> log likelihood: -826824
INFO:lda:<50> log likelihood: -824765
INFO:lda:<60> log likelihood: -823412
INFO:lda:<70> log likelihood: -822317
INFO:lda:<80> log likelihood: -822089
INFO:lda:<90> log likelihood: -821530
INFO:lda:<100> log likelihood: -820496
INFO:lda:<110> log likelihood: -819949
INFO:lda:<120> log likelihood: -820248
INFO:lda:<130> log likelihood: -819988
INFO:lda:<140> log likelihood: -819933
INFO:lda:<150> log likelihood: -818629
INFO:lda:<160> log likelihood: -818887
INFO:lda:<170> log likelihood: -818288
INFO:lda:<180> log likelihood: -818667
INFO:lda:<190> log likelihood: -817431
INFO:lda:<200> log likelihood: -817748
INFO:lda:<210> log likelihood: -817861
INFO:lda:<220> log likelihood: -81

In [58]:

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
review_df=review_df.join(doc_topic)
restaurant=pd.DataFrame()



In [59]:
doc_topic

,0,1,2,3
0,0.304012,0.159979,0.478909,0.057099
1,0.226072,0.116960,0.406011,0.250957
2,0.249401,0.136799,0.510541,0.103258
3,0.178542,0.124689,0.530655,0.166114
4,0.053473,0.306988,0.268641,0.370899
...,...,...,...,...
203,0.499708,0.096671,0.324474,0.079147
204,0.392636,0.430402,0.124224,0.052738
205,0.710833,0.078917,0.131334,0.078917
206,0.170346,0.371105,0.252747,0.205803


In [60]:
review_df

,index,President,Date,Speech,0,1,2,3
0,N,Narendra Modi,2014-01-05 00:00:00,election election movement election movement e...,0.304012,0.159979,0.478909,0.057099
1,N,Narendra Modi,2013-12-22 00:00:00,resent stage party dais ji belief role none ma...,0.226072,0.116960,0.406011,0.250957
2,N,Narendra Modi,2014-02-05 00:00:00,president secretary stage gratitude gathering ...,0.249401,0.136799,0.510541,0.103258
3,N,Narendra Modi,2014-02-02 00:00:00,helicopter saffron sea mela sabha lot time tod...,0.178542,0.124689,0.530655,0.166114
4,september-9-2021-remarks-fighting-covid-19-pan...,Joe Biden,"September 09, 2021",anscript evening battle progress work variant ...,0.053473,0.306988,0.268641,0.370899
...,...,...,...,...,...,...,...,...
203,may-12-1989-commencement-address-texas-am-univ...,George H. W. Bush,"May 12, 1989",anscript welcome luck luck thank thank treat t...,0.499708,0.096671,0.324474,0.079147
204,february-9-1989-address-joint-session-congress,George H. W. Bush,"February 09, 1989",anscript building past hand page history story...,0.392636,0.430402,0.124224,0.052738
205,january-20-1989-inaugural-address,George H. W. Bush,"January 20, 1989",anscript man place history behalf nation word ...,0.710833,0.078917,0.131334,0.078917
206,september-25-1988-debate-michael-dukakis,George H. W. Bush,"September 25, 1988",evening behalf debate campaign panel format no...,0.170346,0.371105,0.252747,0.205803


In [61]:

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    restaurant[topic]=review_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [62]:
restaurant

,President,topic_0,topic_1,topic_2,topic_3
0,Barack Obama,0.422766,0.321377,0.135802,0.120055
1,Bill Clinton,0.457151,0.289974,0.139942,0.112932
2,Donald Trump,0.377005,0.142758,0.145377,0.334859
3,George H. W. Bush,0.508455,0.196652,0.220203,0.074690
4,George W. Bush,0.524657,0.229610,0.182587,0.063146
5,Joe Biden,0.345116,0.197084,0.224326,0.233473
6,Narendra Modi,0.239507,0.134607,0.481529,0.144357
